In [4]:
from typing import Sequence
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "shellhacks-2023-042c64c6e9eb.json"
import google.cloud.texttospeech as tts
from google.oauth2.credentials import Credentials

def text_to_wav(text: str):
    voice_name = "en-US-Studio-M"
    language_code = "-".join(voice_name.split("-")[:2])
    text_input = tts.SynthesisInput(text=text)
    voice_params = tts.VoiceSelectionParams(
        language_code=language_code, name=voice_name
    )
    audio_config = tts.AudioConfig(audio_encoding=tts.AudioEncoding.LINEAR16)

    client = tts.TextToSpeechClient()
    response = client.synthesize_speech(
        input=text_input,
        voice=voice_params,
        audio_config=audio_config,
    )

    filename = f"{voice_name}.wav"
    with open(filename, "wb") as out:
        out.write(response.audio_content)
        print(f'Generated speech saved to "{filename}"')

text_to_wav("Hello there! Chris here, and today, we're diving deep into the world of triangles—specifically, the Pythagorean Theorem. So, first things first. What is the Pythagorean Theorem? Well, it's a way to relate the lengths of the sides of a right triangle. For any right triangle, if you square the lengths of the two shorter sides, a and b, and add them together, you'll get the square of the length of the hypotenuse, c.")

Generated speech saved to "en-US-Studio-M.wav"


In [2]:
from langchain.llms import OpenAI
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from langchain.pydantic_v1 import BaseModel, Field, validator
from typing import List
from langchain import PromptTemplate
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv
load_dotenv()
chat = ChatOpenAI(model="gpt-4")

class Scene(BaseModel):
    visuals: str
    audio: str

class Storyboard(BaseModel):
    scenes: List[Scene]

template="Can you generate a concise storyboard that helps to explain the following concept/answer the provided question in the style of a khan academy video. Use examples including latex, graphs, animations, charts, and more. Make it as visual as possible. Only include enough information to be able to replicate you vision, the bare minimum description. Your storyboard can include anything possible in manim. You should not exceed 5 scenes. You should answer in the following format: {formatting_instructions}"
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template="{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

parser = PydanticOutputParser(pydantic_object=Storyboard)

storyboard = parser.parse(chat(chat_prompt.format_prompt(formatting_instructions=parser.get_format_instructions(), text="Universal Generalization in Discrete Mathematics").to_messages()).content)

In [47]:
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "shellhacks-2023-042c64c6e9eb.json"
import google.cloud.texttospeech as tts
from pydub import AudioSegment
import pprint
from moviepy.editor import *

scene = storyboard.scenes[0]
# set up tts voice
voice_name = "en-US-Studio-M"
language_code = "-".join(voice_name.split("-")[:2])
text_input = tts.SynthesisInput(text=scene.audio)
voice_params = tts.VoiceSelectionParams(
    language_code=language_code, name=voice_name
)
audio_config = tts.AudioConfig(audio_encoding=tts.AudioEncoding.LINEAR16)

# generate audio
client = tts.TextToSpeechClient()
response = client.synthesize_speech(
    input=text_input,
    voice=voice_params,
    audio_config=audio_config,
)

# save audio and get length in seconds
filename = f"{voice_name}.wav"
with open(filename, "wb") as out:
    out.write(response.audio_content)
    print(f'Generated speech saved to "{filename}"')
audio = AudioSegment.from_wav(filename)
duration_seconds = len(audio) / 1000

# generate visuals
scene_template="""Based on the user's prompt, fill out the template below with manim code to generate the desired visuals.

Note that some aspects of manim have changed since you last used it. Here are the changes you need to make:
ShowCreation -> Create

Only return the code.

from manim import *

class VideoVisual(Scene):
def construct(self):
    YOUR CODE HERE

if frame == "main":
    from manim import config

    config.pixel_height = 1080
    config.pixel_width = 1920

    scene = VideoVisual()
    scene.render()"""

scene_chat_prompt = ChatPromptTemplate.from_messages([
    ("system", scene_template),
    ("human", "{prompt}")
])

manim_code = chat(scene_chat_prompt.format_messages(prompt=scene.visuals)).content

pprint.pprint(manim_code)

with open("exec_test.py", 'w') as pyfile:
    pyfile.write(manim_code)

os.system("python exec_test.py")

Generated speech saved to "en-US-Studio-M.wav"
('from manim import *\n'
 '\n'
 'class VideoVisual(Scene):\n'
 '    def construct(self):\n'
 '        title = Text("Universal Generalization in Discrete Mathematics", '
 'color=YELLOW).scale(0.75)\n'
 '        title.to_edge(UP)\n'
 '        self.play(Create(title))\n'
 '\n'
 '        definition = Text("Universal generalization is a valid rule of '
 'inference from the truth \\n of an arbitrary proposition to the truth of the '
 'proposition \\n universally quantified over some domain.", '
 'color=WHITE).scale(0.5)\n'
 '        definition.next_to(title, DOWN)\n'
 '\n'
 '        self.play(Create(definition))\n'
 '        self.wait(3)\n'
 '\n'
 'if __name__ == "__main__":\n'
 '    from manim import config\n'
 '\n'
 '    config.pixel_height = 1080\n'
 '    config.pixel_width = 1920\n'
 '\n'
 '    scene = VideoVisual()\n'
 '    scene.render()')
Manim Community v0.17.3



[09/16/23 02:29:47] INFO     Animation 0 : Partial      ]8;id=436975;file:///Users/owenburns/anaconda3/envs/ShellHacks-2023/lib/python3.11/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=781049;file:///Users/owenburns/anaconda3/envs/ShellHacks-2023/lib/python3.11/site-packages/manim/scene/scene_file_writer.py#527\527]8;;\
                             movie file written in                              
                             '/Users/owenburns/workarea                         
                             s/GPTeacher/cloud_function                         
                             /media/videos/1080p60/part                         
                             ial_movie_files/VideoVisua                         
                             l/1413466013_3754695427_22                         
                             3132457.mp4'                                       


[09/16/23 02:29:48] INFO     Animation 1 : Partial      ]8;id=233533;file:///Users/owenburns/anaconda3/envs/ShellHacks-2023/lib/python3.11/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=130826;file:///Users/owenburns/anaconda3/envs/ShellHacks-2023/lib/python3.11/site-packages/manim/scene/scene_file_writer.py#527\527]8;;\
                             movie file written in                              
                             '/Users/owenburns/workarea                         
                             s/GPTeacher/cloud_function                         
                             /media/videos/1080p60/part                         
                             ial_movie_files/VideoVisua                         
                             l/1672018281_4164062924_16                         
                             58482516.mp4'                                      
[09/16/23 02:29:49] INFO     Animation 2 : Partial      ]8;id=788789;

0

In [48]:
# overlay audio onto video
audio = AudioFileClip("en-US-Studio-M.wav")
video = VideoFileClip("media/videos/1080p60/VideoVisual.mp4")

# Set the audio of the video to the provided audio
final_video = video.set_audio(audio)
final_video.write_videofile(f"final_1.mp4", codec='libx264')


Moviepy - Building video final_1.mp4.
MoviePy - Writing audio in final_1TEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video final_1.mp4



Moviepy - Done !
Moviepy - video ready final_1.mp4


In [45]:
from manim import *

class VideoVisual(Scene):
    def construct(self):
        title = Text("Universal Generalization in Discrete Mathematics", font_size=24)
        title.to_edge(UP)
        self.play(Write(title))

        definition = Text("Universal generalization is a valid rule of inference from the truth of an arbitrary proposition to the truth of the proposition universally quantified over some domain.", font_size=24)
        definition.next_to(title, DOWN, buff=1)
        self.play(Create(definition))

if __name__ == "__main__":
    scene = VideoVisual()
    scene.render()

[09/16/23 02:25:56] INFO     Animation 0 : Partial movie file written in                   ]8;id=12544;file:///Users/owenburns/anaconda3/envs/ShellHacks-2023/lib/python3.11/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=288058;file:///Users/owenburns/anaconda3/envs/ShellHacks-2023/lib/python3.11/site-packages/manim/scene/scene_file_writer.py#527\527]8;;\
                             '/Users/owenburns/workareas/GPTeacher/cloud_function/media/vi                         
                             deos/700p60/partial_movie_files/VideoVisual/2643061277_130877                         
                             7899_223132457.mp4'                                                                   

[09/16/23 02:25:58] INFO     Animation 1 : Partial movie file written in                   ]8;id=107332;file:///Users/owenburns/anaconda3/envs/ShellHacks-2023/lib/python3.11/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=68278;file:///Users/owenburns/anaconda3/envs/ShellHacks-2023/lib/python3.11/site-packages/manim/scene/scene_file_writer.py#527\527]8;;\
                             '/Users/owenburns/workareas/GPTeacher/cloud_function/media/vi                         
                             deos/700p60/partial_movie_files/VideoVisual/3436429281_280977                         
                             3359_4114832126.mp4'                                                                  

                    INFO     Combining to Movie file.                                      ]8;id=78205;file:///Users/owenburns/anaconda3/envs/ShellHacks-2023/lib/python3.11/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=351287;file:///Users/owenburns/anaconda3/envs/ShellHacks-2023/lib/python3.11/site-packages/manim/scene/scene_file_writer.py#617\617]8;;\

/Users/owenburns/workareas/GPTeacher/cloud_function/media/videos/2700p60/VideoVisual.mp4: No such file or directory


                    INFO                                                                   ]8;id=65280;file:///Users/owenburns/anaconda3/envs/ShellHacks-2023/lib/python3.11/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=820635;file:///Users/owenburns/anaconda3/envs/ShellHacks-2023/lib/python3.11/site-packages/manim/scene/scene_file_writer.py#736\736]8;;\
                             File ready at                                                                         
                             '/Users/owenburns/workareas/GPTeacher/cloud_function/media/vi                         
                             deos/2700p60/VideoVisual.mp4'                                                         
                                                                                                                   

                    INFO     Rendered VideoVisual                                                      ]8;id=683528;file:///Users/owenburns/anaconda3/envs/ShellHacks-2023/lib/python3.11/site-packages/manim/scene/scene.py\scene.py]8;;\:]8;id=32853;file:///Users/owenburns/anaconda3/envs/ShellHacks-2023/lib/python3.11/site-packages/manim/scene/scene.py#241\241]8;;\
                             Played 2 animations                                                                   